# Reproducing Dublin pubs example

[Levi Wolf's Guinness delivery example](https://gist.github.com/ljwolf/e5927ab8c859ed477f496329c1ce19fc#file-guinness-py)

In [1]:
import geopandas as gpd
import pandas, numpy

In [2]:
import pandas

In [3]:
import sys
sys.path.insert(0, '/home/dylan/projects/gsoc2025/spopt/') # not the published spopt
import spopt

In [4]:
print(spopt.__file__)

/home/dylan/projects/gsoc2025/spopt/spopt/__init__.py


In [5]:
from spopt.route import engine, heuristic, utils

In [6]:
spopt.route.utils?

Type:        module
String form: <module 'spopt.route.utils' from '/home/dylan/projects/gsoc2025/spopt/spopt/route/utils.py'>
File:        ~/projects/gsoc2025/spopt/spopt/route/utils.py
Docstring:   <no docstring>

In [7]:
dir(spopt.route.engine)

['_OSRM_DATABASE_FILE',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_build_route_table_http',
 '_build_route_table_pyosrm',
 '_build_specific_route_http',
 '_build_specific_route_pyosrm',
 '_create_route_request',
 'build_route_table',
 'build_specific_route',
 'geopandas',
 'has_bindings',
 'metrics',
 'numpy',
 'os',
 'requests',
 'shapely',
 'warnings']

In [9]:
engine.build_specific_route?

Signature:
engine.build_specific_route(
    waypoints,
    port=5000,
    http=True,
    return_durations=True,
    database_path='',
)
Docstring:
Build a route over the road network from each waypoint to each other waypoint. If the routing engine is not found, this builds straight-line 
routes, and measures their duration as a nonlinear function of the 
haversine distance between input points. 
File:      ~/projects/gsoc2025/spopt/spopt/route/engine.py
Type:      function

Reproduce the guinness example

In [10]:
from spopt.route.heuristic import LastMile
from pyvrp import stop

In [11]:
trucks = pandas.DataFrame(
    [['big', 'lng',      2000,    280, .004,  .50, 5],
     ['big', 'electric', 2000,    480, .002,  .50, 5],
     ['med', 'lng',      800, 280*.66, .0001, .63, 10],
     ['med', 'electric', 800, 480*.66, .004,  .50, 10],
     ['smo', 'lng',      400, 280*0.4, .002,  .50, 20],
     ['smo', 'electric', 400, 480*0.4, .0001, .63, 20],
     ],
     columns = [
         'namesize', 'namefuel', 'capacity', 
         'fixed_cost', 'cost_per_meter', 'cost_per_minute', 'n_truck'
         ]
)

In [12]:
dublin_pubs = gpd.read_file('/home/dylan/projects/gsoc2025/spopt/notebooks/gsoc2025/data/dublin-pubs.geojson')

In [13]:
dublin_pubs.shape

(408, 9)

In [14]:
gdf = dublin_pubs
gdf = gdf.iloc[:407]

In [15]:
gdf['longitude'] = gdf.geometry.x
gdf['latitude'] = gdf.geometry.y

/home/dylan/mambaforge/envs/gsoc/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/dylan/mambaforge/envs/gsoc/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [29]:
depot = gdf.iloc[0] #unsure where the depot is?
clients = gdf.iloc[1:] 

In [30]:
depot

fill-opacity                            NaN
review              Smallest Pub In Ireland
name                          Dawson Lounge
marker-symbol                          beer
stroke                                 None
stroke-width                            NaN
stroke-opacity                          NaN
fill                                   None
geometry          POINT (-6.25886 53.33972)
longitude                          -6.25886
latitude                           53.33972
Name: 0, dtype: object

In [31]:
clients.head()

,fill-opacity,review,name,marker-symbol,stroke,stroke-width,stroke-opacity,fill,geometry,longitude,latitude
1,NaN,"Rugger Bar, but not bad",Paddy Cullens,beer,None,NaN,NaN,None,POINT (-6.23036 53.329),-6.23036,53.32900
2,NaN,Can't Remember. Possibly Drunk,Crowes Pub,beer,None,NaN,NaN,None,POINT (-6.23048 53.32905),-6.23048,53.32905
3,NaN,Decent Boozer. On the Lansdowne route.,The Wellington,beer,None,NaN,NaN,None,POINT (-6.24527 53.33387),-6.24527,53.33387
4,NaN,Cock Pub. Think its closed.,Searsons,beer,None,NaN,NaN,None,POINT (-6.24303 53.33332),-6.24303,53.33332
5,NaN,Good Pub for pre Ireland match drinks. Always ...,Scruffy Murphys,beer,None,NaN,NaN,None,POINT (-6.24276 53.33788),-6.24276,53.33788


In [36]:
clients.explore()

In [32]:
clients['demand'] = 1
clients['supply'] = 1

/home/dylan/mambaforge/envs/gsoc/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/dylan/mambaforge/envs/gsoc/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [33]:
# clients = clients.set_index('name')

In [34]:
print('initializing model')
m = LastMile(
    depot_location=(depot.longitude.item(), depot.latitude.item()),
    depot_open=pandas.Timestamp("2030-01-02 07:00:00"),
    depot_close=pandas.Timestamp("2030-01-02 20:00:00"),
    depot_name=depot['name'],
)
print("adding clients")

m.add_clients(
    locations = clients.geometry, 
    delivery = clients.demand,
    pickup = clients.supply,
    time_windows=None,
    service_times=(numpy.log(clients.demand)**2).astype(int)
)
print("adding trucks")
m.add_trucks_from_frame(
    trucks, 
)
m.solve(stop=stop.MaxRuntime(60))
m.write_result("./guinness")

initializing model
adding clients
adding trucks


/home/dylan/projects/gsoc2025/spopt/spopt/route/engine.py:31: UserWarning: Failed to connect to routing engine... using haversine distance and (d/500)**.75 for durations
  warnings.warn(


KeyError: '1'

In [ ]:
import pyvrp

In [ ]:
dir(pyvrp.Model)